# Lightning Module:

In [1]:
import os
from torch import optim, nn, utils, Tensor
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import pytorch_lightning as pl

# define any number of nn.Modules (or use your current ones)
encoder = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(64, 3))
decoder = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28))

# define the LightningModule
class LitAutoEncoder(pl.LightningModule):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = nn.functional.mse_loss(x_hat, x)
        # Logging to TensorBoard by default
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


# init the autoencoder
autoencoder = LitAutoEncoder(encoder, decoder)

# Dataloader:

In [2]:
# setup data
dataset = MNIST(os.getcwd(), download=True, transform=ToTensor())
train_loader = utils.data.DataLoader(dataset)

100%|██████████| 9912422/9912422 [00:01<00:00, 8781165.74it/s] 


Extracting /media/warmachine/DBDISK/SSFocus/MNIST/raw/train-images-idx3-ubyte.gz to /media/warmachine/DBDISK/SSFocus/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 7000040.09it/s]

Extracting /media/warmachine/DBDISK/SSFocus/MNIST/raw/train-labels-idx1-ubyte.gz to /media/warmachine/DBDISK/SSFocus/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 6527880.70it/s]


Extracting /media/warmachine/DBDISK/SSFocus/MNIST/raw/t10k-images-idx3-ubyte.gz to /media/warmachine/DBDISK/SSFocus/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 1261875.13it/s]


Extracting /media/warmachine/DBDISK/SSFocus/MNIST/raw/t10k-labels-idx1-ubyte.gz to /media/warmachine/DBDISK/SSFocus/MNIST/raw



# Trainer Fit:

In [5]:
# train the model (hint: here are some helpful Trainer arguments for rapid idea iteration)
trainer = pl.Trainer(limit_train_batches=100, max_epochs=100)
trainer.fit(model=autoencoder, train_dataloaders=train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 50.4 K
1 | decoder | Sequential | 51.2 K
---------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407     Total estimated model params size (MB)


Epoch 99: 100%|██████████| 100/100 [00:00<00:00, 538.80it/s, v_num=2]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 100/100 [00:00<00:00, 527.56it/s, v_num=2]


# Use the model:

In [7]:
# load checkpoint
checkpoint = "./lightning_logs/version_0/checkpoints/epoch=0-step=100.ckpt"
autoencoder = LitAutoEncoder.load_from_checkpoint(checkpoint, encoder=encoder, decoder=decoder)

# choose your trained nn.Module
encoder = autoencoder.encoder
encoder.eval()

# embed 4 fake images!
fake_image_batch = Tensor(4, 28 * 28)
embeddings = encoder(fake_image_batch.cuda())
print("⚡" * 20, "\nPredictions (4 image embeddings):\n", embeddings, "\n", "⚡" * 20)

⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡ 
Predictions (4 image embeddings):
 tensor([[ 8.8991e+34, -1.7888e+36, -2.9100e+36],
        [-1.0011e+36, -1.6726e+36,  2.0514e+35],
        [-8.1809e+30, -1.0769e+31, -1.8467e+31],
        [        nan,         nan,         nan]], device='cuda:0',
       grad_fn=<AddmmBackward0>) 
 ⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡
